In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Libraries for Recommendation System
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import os
df_movies = pd.read_csv("movies.csv", encoding='cp949', sep=',', on_bad_lines='skip')#, encoding='cp949', sep='\t'
df_ratings = pd.read_csv("ratings.csv")

In [2]:
df_ratings.drop('timestamp', axis=1, inplace=True)
df_ratings.head()

,userId,movieId,rating
0,1.0,31.0,2.5
1,1.0,1029.0,3.0
2,1.0,1061.0,3.0
3,1.0,1129.0,2.0
4,1.0,1172.0,4.0


In [3]:
df_movies.drop('genres', axis=1, inplace=True)
df_movies.drop('Unnamed: 3', axis=1, inplace=True)
df_movies.head()

,movieId,title
0,1.0,Toy Story (1995)
1,2.0,Jumanji (1995)
2,3.0,Grumpier Old Men (1995)
3,4.0,Waiting to Exhale (1995)
4,5.0,Father of the Bride Part II (1995)


In [4]:
data = pd.merge(df_movies,df_ratings)

In [5]:
# pivot ratings into movie features
df_movie_features = data.pivot_table(index = ["title"],columns = ["userId"],values = "rating").fillna(0)
df_movie_features.head()

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,662.0,663.0,664.0,665.0,666.0,667.0,668.0,669.0,670.0,671.0
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Neath the Arizona Skies (1934),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# We choose random movie.
query_index = np.random.choice(df_movie_features.shape[0])
print("Choosen Movie is: ",df_movie_features.index[query_index])

Choosen Movie is:  Solace (2015)


In [7]:
from scipy.sparse import csr_matrix
# convert dataframe of movie features to scipy sparse matrix
user_movie_table_matrix = csr_matrix(df_movie_features.values)

In [8]:
df_movie_features.head()

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,662.0,663.0,664.0,665.0,666.0,667.0,668.0,669.0,670.0,671.0
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Neath the Arizona Skies (1934),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_movie_table_matrix)
distances, indices = model_knn.kneighbors(df_movie_features.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 6)

In [10]:
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(df_movie_features.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(df_movie_features.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["movie"].iloc[i], recommend["distance"].iloc[i]))

Recommendations for Solace (2015):

0: The Longest Week (2014), with distance of 0.0
1: Someone Marry Barry (2014), with distance of 0.0
2: Rookie, The (1990), with distance of 0.0
3: Elektra Luxx (2010), with distance of 0.0
4: Call, The (2013), with distance of 0.0


In [66]:
#k-means

In [11]:
#All the header files required for the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
df_movies = pd.read_csv("movies.csv", encoding='cp949', sep=',', on_bad_lines='skip')
df_ratings = pd.read_csv("ratings.csv")

In [12]:
df_ratings.drop('timestamp', axis=1, inplace=True)
df_ratings.head()

,userId,movieId,rating
0,1.0,31.0,2.5
1,1.0,1029.0,3.0
2,1.0,1061.0,3.0
3,1.0,1129.0,2.0
4,1.0,1172.0,4.0


In [13]:
df_movies.drop('Unnamed: 3', axis=1, inplace=True)
df_movies.head()

,movieId,title,genres
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,3.0,Grumpier Old Men (1995),Comedy|Romance
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5.0,Father of the Bride Part II (1995),Comedy


In [14]:
#Merging both the dataframes based on MovieId
data = pd.merge(df_ratings , df_movies , how='outer', on='movieId')
data.head()

,userId,movieId,rating,title,genres
0,1.0,31.0,2.5,Dangerous Minds (1995),Drama
1,7.0,31.0,3.0,Dangerous Minds (1995),Drama
2,31.0,31.0,4.0,Dangerous Minds (1995),Drama
3,32.0,31.0,4.0,Dangerous Minds (1995),Drama
4,36.0,31.0,3.0,Dangerous Minds (1995),Drama


In [15]:
# Because of merging some null values are created
data.isnull().sum()

userId     60
movieId     1
rating     60
title       4
genres      4
dtype: int64

In [16]:
#WE simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [17]:
data.isnull().sum()

userId     0
movieId    0
rating     0
title      0
genres     0
dtype: int64

In [18]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = data.genres
a = list()
for i in x:
    abc = i
    a.append(abc.split("|"))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,userId,movieId,rating,title,genres,Drama,Animation,Thriller,Action,Fantasy,...,Horror,Documentary,Film-Noir,Musical,Sci-Fi,Romance,Western,War,(no genres listed),Charlie Brown (and Don't Come Back!) (1980)
0,1.0,31.0,2.5,Dangerous Minds (1995),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,31.0,3.0,Dangerous Minds (1995),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['genres'].str.contains(i), i] = 1
data.head(2)

C:\Users\LG-PC\AppData\Local\Temp\ipykernel_27728\4275147318.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data.loc[data['genres'].str.contains(i), i] = 1


,userId,movieId,rating,title,genres,Drama,Animation,Thriller,Action,Fantasy,...,Horror,Documentary,Film-Noir,Musical,Sci-Fi,Romance,Western,War,(no genres listed),Charlie Brown (and Don't Come Back!) (1980)
0,1.0,31.0,2.5,Dangerous Minds (1995),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,31.0,3.0,Dangerous Minds (1995),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Now there is no use of genre 
# Since we have movie id so there is no need for movie names as well
data = data.drop(['genres','title'],axis =1)
data.head()

,userId,movieId,rating,Drama,Animation,Thriller,Action,Fantasy,Adventure,Comedy,...,Horror,Documentary,Film-Noir,Musical,Sci-Fi,Romance,Western,War,(no genres listed),Charlie Brown (and Don't Come Back!) (1980)
0,1.0,31.0,2.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,31.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,31.0,31.0,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,32.0,31.0,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36.0,31.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data.columns

Index(['userId', 'movieId', 'rating', 'Drama', 'Animation', 'Thriller',
       'Action', 'Fantasy', 'Adventure', 'Comedy', 'Mystery', 'Crime',
       'Children', 'Horror', 'Documentary', 'Film-Noir', 'Musical', 'Sci-Fi',
       'Romance', 'Western', 'War', '(no genres listed)',
       ' Charlie Brown (and Don't Come Back!) (1980)'],
      dtype='object')

In [26]:
# Because of merging some null values are created
data.isnull().sum()

userId                                          0
movieId                                         0
rating                                          0
Drama                                           0
Animation                                       0
Thriller                                        0
Action                                          0
Fantasy                                         0
Adventure                                       0
Comedy                                          0
Mystery                                         0
Crime                                           0
Children                                        0
Horror                                          0
Documentary                                     0
Film-Noir                                       0
Musical                                         0
Sci-Fi                                          0
Romance                                         0
Western                                         0


In [28]:
#By different meathods I found 8 cluster are better 
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans()

In [29]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

89433    0
1180     0
40015    0
55584    5
45798    0
38341    0
37591    0
91342    5
93256    2
69482    5
Name: Cluster, dtype: int32

In [30]:
data['Cluster'].value_counts()

0    62089
5    22650
7     4457
2     3078
3     2863
1     2394
6     1915
4      555
Name: Cluster, dtype: int64

In [31]:
data.head()

,userId,movieId,rating,Drama,Animation,Thriller,Action,Fantasy,Adventure,Comedy,...,Documentary,Film-Noir,Musical,Sci-Fi,Romance,Western,War,(no genres listed),Charlie Brown (and Don't Come Back!) (1980),Cluster
0,1.0,31.0,2.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,31.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,31.0,31.0,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,32.0,31.0,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36.0,31.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie 
# the Cluster which occurs maximum number of times is alloted to the movie  
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['movieId'][d[0]],int(b.idxmax())]
    e.append(c)

In [33]:
data.groupby("movieId").apply(lambda x: fi(x))

""


In [34]:
e = pd.DataFrame(e)

In [35]:
e.head()

,0,1
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,0


In [37]:
# I Dont know why always the column name shift according to its will :(
# Here just the column names are swapped
e.rename(columns = {0:'movieId',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)
e

,movieId,Cluster
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,0
...,...,...
9059,161944.0,4
9060,162376.0,4
9061,162542.0,4
9062,162672.0,4


In [38]:
new_data = pd.merge(e , df_movies , how='outer', on='movieId')

In [39]:
# These were the movies we deleted while merging the file  
new_data.isnull().sum()

movieId     1
Cluster    60
title       1
genres      1
dtype: int64

In [40]:
# We can delete the movies but I just label them randomly :)
new_data.fillna(random.randint(0,8),inplace=True)

In [41]:
new_data.isnull().sum()

movieId    0
Cluster    0
title      0
genres     0
dtype: int64

In [47]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['movieId'][i] , new_data['title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.MovieID == l]

In [48]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
        print('-----------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
            print('----------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
df_movies   = pd.read_csv("movies.csv", encoding='cp949', sep=',', on_bad_lines='skip')#, encoding='cp949', sep='\t'
df_ratings   = pd.read_csv("ratings.csv")

In [52]:
df_user_movie_ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [53]:
df_user_movie_ratings.head()

movieId,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,161084.0,161155.0,161594.0,161830.0,161918.0,161944.0,162376.0,162542.0,162672.0,163949.0
userId,,,,,,,,,,,,,,,,,,,,,
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#matrix is the value of pivot_table as numpy matrix
matrix = df_user_movie_ratings.to_numpy()
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [55]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [56]:
matrix.shape

(672, 9067)

In [57]:
user_ratings_mean.shape

(672,)

In [58]:
matrix_user_mean.shape

(672, 9067)

In [59]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

movieId,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,161084.0,161155.0,161594.0,161830.0,161918.0,161944.0,162376.0,162542.0,162672.0,163949.0
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
2,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,...,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227,-0.029227
3,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,...,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073,-0.020073
4,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,...,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827,-0.097827


In [60]:
from scipy.sparse.linalg import svds
from scipy.linalg import svd

In [61]:
U, sigma, Vt = svds(matrix_user_mean, k =12)

In [62]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(672, 12)
(12,)
(12, 9067)


In [63]:
sigma = np.diag(sigma)

In [64]:
sigma[0]

array([105.72438591,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [65]:
#Recover to original matrix
#Apply user average rating to it
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [66]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds.head()

movieId,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,161084.0,161155.0,161594.0,161830.0,161918.0,161944.0,162376.0,162542.0,162672.0,163949.0
0,2.937474e-17,-5.660852e-16,-3.457961e-16,-2.252505e-16,3.292947e-18,-3.146645e-16,-2.161049e-16,-4.764541e-16,-2.356419e-17,-1.035417e-16,...,4.301639e-17,2.564408e-17,3.753720e-17,3.351167e-17,3.558013e-17,-1.734477e-17,4.161843e-17,2.620105e-17,2.747053e-17,5.665803e-17
1,-4.879758e-03,-7.968646e-02,2.177819e-02,-1.383648e-02,-5.869482e-03,-2.887724e-02,3.237051e-02,7.147544e-04,-4.427359e-03,-5.218537e-03,...,-4.322936e-03,-4.351450e-03,1.047879e-02,-4.255512e-03,-3.943389e-03,-5.673536e-03,1.815807e-02,-5.574443e-03,-5.296569e-03,-3.766113e-03
2,-4.472953e-04,1.428475e+00,1.608840e+00,5.294863e-01,1.682786e-01,5.208189e-01,1.107491e+00,5.297333e-01,8.937498e-02,2.962755e-01,...,1.322683e-02,-2.274878e-03,2.067939e-02,-5.245791e-03,-7.645038e-03,-2.101788e-02,3.124273e-02,-9.575828e-04,-7.534678e-04,2.690096e-02
3,1.126053e-02,9.772596e-01,3.969741e-01,3.015839e-04,2.744315e-02,2.128984e-02,1.414716e-01,-5.712665e-02,3.163079e-02,-1.253570e-02,...,2.760222e-03,4.905090e-03,-1.419007e-02,-2.534059e-04,-6.010376e-03,-3.190976e-03,-2.691538e-02,1.463496e-02,1.328519e-02,-5.740090e-03
4,-1.373184e-02,1.870879e+00,1.169988e+00,2.522059e-01,9.483786e-02,-1.817143e-01,-5.119284e-01,-2.780898e-02,-1.430758e-01,1.325995e-02,...,2.641594e-02,-2.724122e-02,5.468925e-02,1.845138e-02,3.454299e-02,-3.573594e-02,8.889979e-02,-1.936081e-02,-1.710922e-02,6.656371e-02


In [67]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [68]:
already_rated, predictions = recommend_movies(df_svd_preds, 330, df_movies, df_ratings, 10)

In [69]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres,Unnamed: 3
47,330.0,1094.0,5.0,948577200.0,"Crying Game, The (1992)",Drama|Romance|Thriller,NaN
14,330.0,213.0,5.0,948577039.0,Burnt by the Sun (Utomlyonnye solntsem) (1994),Drama,NaN
34,330.0,527.0,5.0,948577164.0,Schindler's List (1993),Drama|War,NaN
45,330.0,1035.0,5.0,948574126.0,"Sound of Music, The (1965)",Musical|Romance,NaN
48,330.0,1172.0,5.0,948575236.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN
53,330.0,1249.0,5.0,948576522.0,"Femme Nikita, La (Nikita) (1990)",Action|Crime|Romance|Thriller,NaN
27,330.0,373.0,5.0,948576522.0,Red Rock West (1992),Thriller,NaN
58,330.0,1428.0,5.0,948731410.0,Angel Baby (1995),Drama,NaN
61,330.0,1575.0,5.0,948577442.0,Gabbeh (1996),Drama,NaN
67,330.0,1683.0,5.0,948578017.0,"Wings of the Dove, The (1997)",Drama|Romance,NaN


In [70]:
predictions

,movieId,title,genres,Unnamed: 3,Predictions
214,260.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,NaN,1.679814
604,780.0,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller,NaN,1.444927
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,1.244859
913,1210.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,NaN,1.226031
522,648.0,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,NaN,1.222761
575,733.0,"Rock, The (1996)",Action|Adventure|Thriller,NaN,1.101635
901,1196.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,NaN,1.093068
497,608.0,Fargo (1996),Comedy|Crime|Drama|Thriller,NaN,1.069377
577,736.0,Twister (1996),Action|Adventure|Romance|Thriller,NaN,0.939855
1044,1356.0,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,NaN,0.919837
